In [1]:
!pip install flask flask-ngrok transformers torch

In [6]:
!pkill -f ngrok


In [7]:
!pgrep -af ngrok

In [8]:
!ngrok authtoken 2uq6RFdCwvzFOIb1lebHE7sEfBP_6Qkvpv1kCFgLrCCnGYq52


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
import torch
import pandas as pd
from flask import Flask, render_template_string, request, send_file
from transformers import T5Tokenizer, T5ForConditionalGeneration
from pyngrok import ngrok
from jira import JIRA

# JIRA Configuration
JIRA_URL = "https://narenreddyminukuri.atlassian.net"
JIRA_EMAIL = "narenreddyminukuri@gmail.com"
JIRA_API_TOKEN = "ATATT3xFfGF0k8Oy963gN-mIQugAzXjTNtRa_9NbH-k_mxZJVyn1Y1ZyQwFQFtiPB5R1fiP8sZ3PKCzcGMFu8oabg7iskkY5GWuep24s0GbmbTOfApAmZtp7c8Ca31sKUPq8JNnufXqtkD4i1X4D_4XAi8kb29bIIP-O1rziq1EfoNHD-YfCyv4=F249F0FC"
options = {"server": JIRA_URL}
jira = JIRA(options, basic_auth=(JIRA_EMAIL, JIRA_API_TOKEN))

# Initialize Flask app
app = Flask(__name__)

# Load the trained model
model_name = "/content/drive/MyDrive/test_case_model"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_feature_from_jira(issue_key):
    issue = jira.issue(issue_key)
    return issue.fields.description if issue.fields.description else "No description found."

def generate_unique_test_cases(feature_description, num_cases=8, max_length=512, max_attempts=10):
    input_text = "generate test case and expected outcome: " + feature_description
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    unique_test_cases = set()
    attempts = 0

    while len(unique_test_cases) < num_cases and attempts < max_attempts:
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_length=max_length,
                num_return_sequences=num_cases,
                temperature=0.7,
                num_beams=10,
                early_stopping=True
            )
        for out in output:
            decoded_tc = tokenizer.decode(out, skip_special_tokens=True)
            if decoded_tc not in unique_test_cases:
                unique_test_cases.add(decoded_tc)
        attempts += 1

    return list(unique_test_cases)[:num_cases]

def process_test_case_output(test_case_output):
    if "|" in test_case_output:
        test_case, expected_outcome = test_case_output.split("|", 1)
        return test_case.strip(), expected_outcome.strip()
    else:
        return test_case_output.strip(), "Expected outcome not generated, manually verify."

def save_to_excel(test_cases):
    df = pd.DataFrame(test_cases)
    file_path = "generated_test_cases.xlsx"
    df.to_excel(file_path, index=False)
    return file_path

@app.route('/', methods=['GET', 'POST'])
def index():
    test_cases = []
    feature_desc = ""
    input_source = "manual"
    issue_key = ""

    if request.method == 'POST':
        input_source = request.form['input_source']
        if input_source == "jira":
            issue_key = request.form['jira_key']
            feature_desc = get_feature_from_jira(issue_key)
        else:
            feature_desc = request.form['feature_desc']

        generated_cases = generate_unique_test_cases(feature_desc)
        test_cases = [
            {"no": i + 1, "test_case": process_test_case_output(tc)[0],
             "expected_outcome": process_test_case_output(tc)[1]}
            for i, tc in enumerate(generated_cases)
        ]

    html_template = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Test Case Generator</title>
        <style>
            body { background-color: white; color: #a6192e; text-align: center; padding: 20px; font-family: Arial, sans-serif; }
            textarea, button, input, select { padding: 10px; margin: 10px; width: 80%; }
            textarea { height: 120px; font-size: 16px; }
            button { background-color: #a6192e; color: white; border: none; font-weight: bold; cursor: pointer; padding: 12px; }
            table { width: 80%; margin: 20px auto; border-collapse: collapse; }
            th, td { border: 1px solid #a6192e; padding: 10px; text-align: left; }
            th { background-color: #a6192e; color: white; }
            .export-button { margin-top: 20px; }
        </style>
        <script>
            function toggleInput() {
                var source = document.getElementById('input_source').value;
                document.getElementById('jira_section').style.display = source === 'jira' ? 'block' : 'none';
                document.getElementById('feature_desc').readOnly = source === 'jira';
            }
        </script>
    </head>
    <body>
        <h2>Test Case Generator</h2>
        <form method="post">
            <label for="input_source"><b>Select Input Source:</b></label><br>
            <select id="input_source" name="input_source" onchange="toggleInput()">
                <option value="manual" {% if input_source=='manual' %}selected{% endif %}>Manual</option>
                <option value="jira" {% if input_source=='jira' %}selected{% endif %}>JIRA</option>
            </select><br>
            <div id="jira_section" style="display:none;">
                <input id="jira_key" name="jira_key" type="text" placeholder="Enter JIRA Issue Key" value="{{ issue_key }}"><br>
            </div>
            <textarea id="feature_desc" name="feature_desc">{{ feature_desc }}</textarea><br>
            <button type="submit">Generate Test Cases</button>
        </form>
        {% if test_cases %}
            <h3>Generated Test Cases & Expected Outcomes</h3>
            <table>
                <tr>
                    <th>Test Case No.</th>
                    <th>Test Case</th>
                    <th>Expected Outcome</th>
                </tr>
                {% for tc in test_cases %}
                <tr>
                    <td>{{ tc.no }}</td>
                    <td>{{ tc.test_case }}</td>
                    <td>{{ tc.expected_outcome }}</td>
                </tr>
                {% endfor %}
            </table>
            <form action="/export" method="get">
                <button class="export-button" type="submit">Export to Excel</button>
            </form>
        {% endif %}
    </body>
    </html>
    """
    return render_template_string(html_template, test_cases=test_cases, feature_desc=feature_desc, input_source=input_source, issue_key=issue_key)

@app.route('/export')
def export():
    test_cases = request.args.getlist('test_cases')
    file_path = save_to_excel(test_cases)
    return send_file(file_path, as_attachment=True)

if __name__ == '__main__':
    port = 5001
    public_url = ngrok.connect(port).public_url
    print(f"Public URL: {public_url}")
    app.run(host='0.0.0.0', port=port)

Public URL: https://9964-34-66-105-97.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.28.0.12:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 10:52:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 10:52:58] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 10:53:25] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 10:53:28] "GET /export HTTP/1.1" 200 -
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWa